In [53]:
from openai import OpenAI
from openai.types.fine_tuning import SupervisedMethod, SupervisedHyperparameters
api_key="<put your key>"
client = OpenAI(api_key=api_key)

In [57]:
import time

def wait_for_finetuning_jobs(job_ids, api_key, poll_interval=60):
    client = OpenAI(api_key=api_key)
    finished_statuses = {"succeeded", "failed", "cancelled"}
    
    while True:
        statuses = []
        for job_id in job_ids:
            job = client.fine_tuning.jobs.retrieve(job_id)
            statuses.append(job.status)
        
        if all(status in finished_statuses for status in statuses):
            print("All jobs finished.")
            return statuses
        time.sleep(poll_interval)

In [ ]:
number_of_iterations = 5 #please change to required number
jobs = [None] * number_of_iterations

for i in range(number_of_iterations):
    job = client.fine_tuning.jobs.create(
        training_file="<put training file id>",
        validation_file="<put validation file id>",
        model="gpt-4o-mini-2024-07-18",
        method={
        "type": "supervised",
        "supervised": SupervisedMethod(
          hyperparameters=SupervisedHyperparameters(
            n_epochs=3,
            batch_size=1,
            learning_rate_multiplier=1.8,
          )
        )
      }
    )
    print(job.id)
    jobs[i] = job.id
    
wait_for_finetuning_jobs(jobs, api_key, poll_interval=60)

In [50]:
def get_output_model_name(job_id, api_key):
    client = OpenAI(api_key=api_key)
    job = client.fine_tuning.jobs.retrieve(job_id)
    return job.fine_tuned_model


In [ ]:
import subprocess
import os
from concurrent.futures import ThreadPoolExecutor
from pathlib import Path
import re
import datetime

task_script = "ArSAS_GPT4o_ZeroShot*" #you can change it to "ArSarcasm_GPT4o_ZeroShot*" or "ASND_GPT4o_ZeroShot*" depending on the task
models = [get_output_model_name(x,api_key) for x in jobs]
OPENAI_API_KEY = api_key
WORKING_DIR = "./"
LOGS_DIR = os.path.join(WORKING_DIR, "logs")

def safe_folder_name(model_name):
    date_str = datetime.datetime.now().strftime('%Y%m%d')
    base = re.sub(r'[^a-zA-Z0-9._-]', '_', model_name)
    return f"{base}_{date_str}"

def run_bench(model):
    folder_name = safe_folder_name(model)
    log_file = os.path.join(LOGS_DIR, f"{folder_name}.log")

    env = os.environ.copy()
    env["OPENAI_API_KEY"] = OPENAI_API_KEY
    env["OPENAI_MODEL"] = model

    #make sure you downloaded llmebench from https://github.com/qcri/LLMeBench
    cmd = [
        "python", "-m", "llmebench",
        "--filter", task_script,
        "assets/", folder_name
    ]

    with open(log_file, "w") as logf:
        logf.write(f"Running in: {WORKING_DIR}\n")
        logf.write(f"Model: {model}\n")
        logf.flush()
        process = subprocess.Popen(
            cmd, env=env, cwd=WORKING_DIR,
            stdout=logf, stderr=subprocess.STDOUT
        )
        process.wait()
        logf.write(f"\nProcess exited with code {process.returncode}\n")

if __name__ == "__main__":
    with ThreadPoolExecutor(max_workers=len(models)) as executor:
        futures = [executor.submit(run_bench, model) for model in models]
        for f in futures:
            f.result()  # Raises any exceptions that occurred
    print("All jobs finished. Check logs in:", LOGS_DIR)
